In [1]:
import pandas as pd
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, GMapOptions, LinearColorMapper, ColorBar, HoverTool, TapTool, OpenURL
from bokeh.plotting import gmap
from bokeh.transform import transform
from bokeh.layouts import row

API_key = input('Google API Key: ')

Google API Key: 


In [4]:
sight_df = pd.read_csv('data/Sights.csv')
hotel_df = pd.read_csv('data/Hotels.csv')
        
# get the z score of the sight review variable
sight_df['z_sight_review'] = (sight_df['sight_reviews'] - sight_df['sight_reviews'].mean())/sight_df['sight_reviews'].std()
        
# generate the recommendation metric, taking the product of sight rating and the z score of sight review
sight_df['rec_sights'] = sight_df['z_sight_review']*sight_df['sight_rating']
        
# get the z score of the hotel review variable
hotel_df['z_hotel_review'] = (hotel_df['hotel_review'] - hotel_df['hotel_review'].mean())/hotel_df['hotel_review'].std()
        
# generate the recommendation metric, taking the product of hotel rating and the z score of hotel review
hotel_df['rec_hotels'] = hotel_df['z_hotel_review']*hotel_df['hotel_rating']

# create a html file to store the three interactive maps
output_file("data/Hawaii Sights and Hotels.html")

# set the map at Hawaii (with the geometry data provided), using Google terrain map as tile with a zoom level of 7
map_options = GMapOptions(lat = 20.716179, lng = -158.214676, map_type = "terrain", zoom = 7)

# this package takes in ColumnDataSource format, here converts our dataframe to the corresponding format to provide the data sources for plotting
# sort the sight data by the recommendation metric, and select the 20 most recommended sights
source1 = ColumnDataSource(sight_df.sort_values(by = ['rec_sights'], ascending = False)[:20])
source2 = ColumnDataSource(hotel_df)

# plot Recommended Sights with Hawaii Hotels by Price
        
# generate a Hawaii base map by making a request to Google MAP API, and specify the title for the map
p1 = gmap(API_key, map_options, title = "Recommended Sights with Hawaii Hotels by Price")

# create triangle objects to represent the sights, specifying the geometry data as x- and y-axis,
# set the color, size of the triangles, also provide a legend to the map
o1 = p1.triangle(x = "LNG", y = "LAT", size = 9, legend = 'sight', fill_color = "blue", fill_alpha = 0.8, source = source1)
# add the hovertool so that when the mouse cursor is at a sight, the sight name will be displayed
p1.add_tools(HoverTool(renderers = [o1], tooltips = [('Sight', '@sight_name')]))

# specify the color for the hotel circles
color_mapper_price = LinearColorMapper(palette = 'BuPu9', low = hotel_df['hotel_price'].min(), high = hotel_df['hotel_price'].max())
        
# create a color bar to indicate the relationship between color and price
color_bar_price = ColorBar(color_mapper = color_mapper_price, label_standoff = 12, location = (0,0), title = 'Price')
        
# create circle objects to represent the hotels, specifying the geometry data as x- and y-axis,
# set the color, size of the circles, also provide a legend to the map
o2 = p1.circle(x = "LNG", y = "LAT", size = 7, legend = 'hotel', color = transform('hotel_price', color_mapper_price), alpha = 0.8, source = source2)
# add the hovertool so that when the mouse cursor is at a hotel, the hotel name and lowest price will be displayed
p1.add_tools(HoverTool(renderers = [o2], tooltips = [('Hotel', '@hotel_name'),
                                                    ('Price', '@hotel_price')]))

# add a taptool that enable user to click a hotel circle and jump to the hotel webpage in TripAdvisor
url = '@hotel_url'  # specify the url source in our ColumnDataSource
tap = TapTool(callback = OpenURL(url = url))  # specify the open link action for this taptool
        
# avoid the hotel circles being grey out during any interations
o2.selection_glyph = None
o2.nonselection_glyph = None
        
# add the taptool to the plot
p1.add_tools(tap)

# add the color bar next to the plot
p1.add_layout(color_bar_price, 'right')

# similar to the above, plot Recommended Sights with Hawaii Hotels by Popularity Metric       
p2 = gmap(API_key, map_options, title = "Recommended Sights with Hawaii Hotels by Popularity Metric")
o3 = p2.triangle(x = "LNG", y = "LAT", size = 9, legend = 'sight', fill_color = "blue", fill_alpha = 0.8, source = source1)
p2.add_tools(HoverTool(renderers = [o3], tooltips = [('Sight', '@sight_name')]))
color_mapper_pop = LinearColorMapper(palette = 'Viridis9', low = hotel_df['hotel_review'].min(), high = hotel_df['hotel_review'].max())
color_bar_pop = ColorBar(color_mapper = color_mapper_pop, label_standoff = 12, location = (0,0), title = 'Popularity')
o4 = p2.circle(x = "LNG", y = "LAT", size = 7, legend = 'hotel', color = transform('hotel_review', color_mapper_pop), alpha = 0.8, source = source2)
p2.add_tools(HoverTool(renderers = [o4], tooltips = [('Hotel', '@hotel_name'),
                                                    ('Price', '@hotel_price')]))
url = '@hotel_url'
tap = TapTool(callback = OpenURL(url = url))
o4.selection_glyph = None
o4.nonselection_glyph = None
p2.add_tools(tap)
p2.add_layout(color_bar_pop, 'right')
        
# similar to the above, plot Recommended Sights with Hawaii Hotels by Recommendation Metric
p3 = gmap(API_key, map_options, title = "Recommended Sights with Hawaii Hotels by Recommendation Metric")
o5 = p3.triangle(x = "LNG", y = "LAT", size = 9, legend = 'sight', fill_color = "blue", fill_alpha = 0.8, source = source1)
p3.add_tools(HoverTool(renderers = [o5], tooltips = [('Sight', '@sight_name')]))
color_mapper_pop = LinearColorMapper(palette = 'PiYG9', low = hotel_df['rec_hotels'].min(), high = hotel_df['rec_hotels'].max())
color_bar_pop = ColorBar(color_mapper = color_mapper_pop, label_standoff = 12, location = (0,0), title = 'Recommendation')
o6 = p3.circle(x = "LNG", y = "LAT", size = 7, legend = 'hotel', color = transform('rec_hotels', color_mapper_pop), alpha = 0.8, source = source2)
p3.add_tools(HoverTool(renderers = [o6], tooltips = [('Hotel', '@hotel_name'),
                                                    ('Price', '@hotel_price')]))
url = '@hotel_url'
tap = TapTool(callback = OpenURL(url = url))
o6.selection_glyph = None
o6.nonselection_glyph = None
p3.add_tools(tap)
p3.add_layout(color_bar_pop, 'right')

# automatically pop up the interactive plots in the browser, displayed in a row
show(row(p1,p2,p3))
        
# store all data to two csv files
sight_df.to_csv("data/Google_sights_data_complete.csv", index=False)
hotel_df.to_csv("data/Hotel_data_complete.csv", index=False)
 